In [1]:
import os
path = os.getcwd()

if path[1:5] == "Home":
    %run /Home/siv30/wad005/master/GRB/src/read_matlab_file.ipynb #UiB paths
    path_coef = '/Home/siv30/wad005/master/GRB/ASIM/COEFS/BGO/'
else:
    %run /Users/andreas/phys/master/GRB/src/read_matlab_file.ipynb # Mac paths
    path_coef = '/Users/andreas/phys/master/GRB/ASIM/COEFS/BGO/'


all UiB paths loaded
read_matlab_file.ipynb loaded successfully!


In [8]:
import os
import pandas as pd
from datetime import datetime

dfs_coefs = {}
dau_det_list = []

coef_filenames = glob.glob(os.path.join(path_coef, "*.txt"))
coef_filenames.sort()

for i in np.arange(1,5):
    for j in np.arange(1,4):
        # construct filename
        filename = os.path.join(path_coef, f"calibration_coefficients_dau_{i}_det_{j}.txt")
        
        # check if file exists
        if not os.path.isfile(filename):
            print(f"{filename} not found, skipping")
            continue
        
        # read csv file
        df = pd.read_csv(filename, delim_whitespace=True, skiprows=[0])
        
        # check if date and time columns exist
        if "DATE" not in df.columns or "TIME" not in df.columns:
            print(f"{filename} does not contain DATE and TIME columns, skipping")
            continue
        
        # join date and time
        df["datetime"] = df.apply(lambda row: datetime.strptime(row["DATE"] + " " + row["TIME"], "%d-%b-%Y %H:%M:%S"), axis=1)
        
        # drop date and time columns
        df = df.drop(columns=["DATE","TIME"])
        
        # add dataframe to dictionary
        key = f"dau_{i}_det_{j}"
        dfs_coefs[key] = df
        dau_det_list.append(key)

def get_coefs(T0):
    """
    param: T0 <datetime>
    return day_det_coefs <dict> containing a,b,c coefs for all 12 detectors"""
    
    if not isinstance(T0, datetime):
        return None
    
    dau_det_coefs = {}
    
    for i in np.arange(1,5): #generating the output dict containing the coefs
        for j in np.arange(1,4):
            dau_det_coefs["dau_{}_det_{}".format(i,j)] = None

    for dau_det in dau_det_list:
        df = dfs_coefs[dau_det]
        idx = (df['datetime'] - T0).abs().idxmin()  # find index of closest datetime
        row = df.loc[idx]
        dau_det_coefs[dau_det] = {"a":row["a"], "b": row["b"], "c":row["c"]}
    return dau_det_coefs



def convert_erg(erg, dau, det, T0):
    """
    Convert energy values from erg to keV using calibration coefficients
    for each detector.
    """
    keV_list = []
    cal_abc = []
    dau_det_coefs = get_coefs(T0)
    
    for i in range(len(erg)):
        dau_val = dau[i]
        det_val = det[i]
        erg_val = erg[i]
        
        # Retrieve calibration coefficients
        a = dau_det_coefs["dau_{}_det_{}".format(dau_val, det_val)]["a"]
        b = dau_det_coefs["dau_{}_det_{}".format(dau_val, det_val)]["b"]
        c = dau_det_coefs["dau_{}_det_{}".format(dau_val, det_val)]["c"]
        
        # Store calibration coefficients for each detector
        cal_abc.append([a, b, c])
        
        # Convert energy from erg to keV
        keV = round(a * erg_val ** 2 + b * erg_val + c, 4)
        keV_list.append(keV)
        
    return np.array(keV_list), np.array(cal_abc)


In [4]:
print("convert_erg.ipynb loaded successfully!")

convert_erg.ipynb loaded successfully!
